#chapter 1- simple approaches to recommender systems

segment 2 - popularity-based recommenders

In [1]:
import pandas as pd
import numpy as np

In [2]:
frame=pd.read_csv('rating_final.csv')
cuisine=pd.read_csv('chefmozcuisine.csv')

In [3]:
frame.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [4]:
cuisine.head()

,placeID,Rcuisine
0,135110,Spanish
1,135109,Italian
2,135107,Latin_American
3,135106,Mexican
4,135105,Fast_Food


#Recommendation based on counts

In [11]:
rating_count=pd.DataFrame(frame.groupby('placeID')['rating'].count())
rating_count.sort_values('rating',ascending=False).head()

,rating
placeID,
135085,36
132825,32
135032,28
135052,25
132834,25


In [62]:
most_rated_places=pd.DataFrame([135085,132825,135032,135052,132834],index=np.arange(5),columns=['placeID'])
summary=pd.merge(most_rated_places,cuisine,on='placeID')
summary


,placeID,Rcuisine
0,135085,Fast_Food
1,132825,Mexican
2,135032,Cafeteria
3,135032,Contemporary
4,135052,Bar
5,135052,Bar_Pub_Brewery
6,132834,Mexican


In [14]:
cuisine['Rcuisine'].describe()

count         916
unique         59
top       Mexican
freq          239
Name: Rcuisine, dtype: object

#2nd project
# model-based collaborative filtering system
#svd matrix factorization

In [17]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.decomposition import TruncatedSVD

In [18]:
columns=['user_id','item_id','rating','timestamp']
frame=pd.read_csv('ml-100k/u.data',sep='\t',names=columns)
frame.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [66]:
columns=['item_id','movie_title','release date','vedio release date','IMDb url','unknown','Action',
 'Adventure','Animation','Childrens','Comedy','Crime''Documentary','Drama','Ffantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Westren','dfg']
movies = pd.read_csv('ml-100k/u.item',sep='|',names=columns,encoding='latin-1')
movies.head()

,item_id,movie_title,release date,vedio release date,IMDb url,unknown,Action,Adventure,Animation,Childrens,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Westren,dfg
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [77]:
movie_name=movies[['movie_title','item_id']]
combined_movies_data=pd.merge(frame,movie_name, on='item_id')
combined_movies_data.head()

,user_id,item_id,rating,timestamp,movie_title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [79]:
combined_movies_data.groupby('item_id')['rating'].count().sort_values(ascending=False).head()

item_id
50     583
258    509
100    508
181    507
294    485
Name: rating, dtype: int64

In [81]:
Filter =combined_movies_data['item_id']==50
combined_movies_data[Filter]['movie_title'].unique()

array(['Star Wars (1977)'], dtype=object)

# Building a utility matrix

In [84]:
ratings_crosstab=combined_movies_data.pivot_table(values='rating',index='user_id',columns='movie_title',fill_value=0)
ratings_crosstab.head()

movie_title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,2,5,0,0,3,4,0,0,...,0,0,0,5,3,0,0,0,4,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,2,0,0,0,0,4,0,0,...,0,0,0,4,0,0,0,0,4,0


# transpose of matrix

In [85]:
ratings_crosstab.shape

(943, 1664)

In [86]:
X=ratings_crosstab.values.T
X.shape

(1664, 943)


## decomposing the matrix

In [87]:
SVD= TruncatedSVD(n_components=12,random_state=17)
resultant_matrix=SVD.fit_transform(X)
resultant_matrix.shape

(1664, 12)

# Generating a correlaton matrix

In [88]:
corr_mat=np.corrcoef(resultant_matrix)
corr_mat.shape

(1664, 1664)


# isolating star wars from the correlation matrix

In [89]:
movies_names=ratings_crosstab.columns
movies_list=list(movies_names)

star_wars=movies_list.index('Star Wars (1977)')
print(star_wars)

1398


In [91]:
corr_star_wars=corr_mat[star_wars]
corr_star_wars.shape

(1664,)


# recommending a highly correlated movie

In [107]:
list(movies_names[(corr_star_wars<1.0) & (corr_star_wars>0.98)])

['Return of the Jedi (1983)']

# project - content based recomender system

##  nearest neighbor algorithm
### . unsupervised classifier 
### . also known as amemory-based system
### .memorizies instance and then recommends an item(a single instance) based on how quantitatively similer it is to a new ,incoming instance


In [108]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.neighbors import NearestNeighbors    

In [109]:
cars=pd.read_csv('mtcars.csv')
cars.columns=['car_names','mpg','cyl','disp','hp','drat','wt','qsec','vs','am','gear','crab']
cars.head()

,car_names,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,crab
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [123]:
t=[15,300,160,3.2]
X=cars.ix[:,(1,3,4,6)].values
X[0:5]

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


array([[ 21.   , 160.   , 110.   ,   2.62 ],
       [ 21.   , 160.   , 110.   ,   2.875],
       [ 22.8  , 108.   ,  93.   ,   2.32 ],
       [ 21.4  , 258.   , 110.   ,   3.215],
       [ 18.7  , 360.   , 175.   ,   3.44 ]])

In [124]:
nbrs=NearestNeighbors(n_neighbors=1).fit(X)

In [125]:
print(nbrs.kneighbors([t]))

(array([[10.77474942]]), array([[22]], dtype=int64))


In [116]:
cars

,car_names,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,crab
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2
5,Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1
6,Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4
7,Merc 240D,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2
8,Merc 230,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2
9,Merc 280,19.2,6,167.6,123,3.92,3.440,18.30,1,0,4,4
